In [10]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import os
import pandas as pd
from typing import Optional, Tuple
import numpy as np
import json
from tqdm import tqdm
from torchvision.transforms import AutoAugment, AutoAugmentPolicy

In [11]:
compiled_labels_path = 'compiled_labels.csv'
img_dir = './faces'

emotion_list = ['adoration', 'affection', 'aggravation', 'agitation', 'agony', 'alarm', 'alienation', 'amazement', 'amusement', 'anger', 'anguish', 'annoyance', 'anxiety', 'apprehension', 'arousal', 'astonishment', 'attraction', 'bitterness', 'bliss', 'caring', 'cheerfulness', 'compassion', 'contempt', 'contentment', 'defeat', 'dejection', 'delight', 'depression', 'desire', 'despair', 'disappointment', 'disgust', 'dislike', 'dismay', 'displeasure', 'distress', 'dread', 'eagerness', 'ecstasy', 'elation', 'embarrassment', 'enjoyment', 'enthrallment', 'enthusiasm', 'envy', 'euphoria', 'exasperation', 'excitement', 'exhilaration', 'fear', 'ferocity', 'fondness', 'fright', 'frustration', 'fury', 'gaiety', 'gladness', 'glee', 'gloom', 'glumness', 'grief', 'grouchiness', 'grumpiness', 'guilt', 'happiness', 'hate', 'homesickness', 'hope', 'hopelessness', 'horror', 'hostility', 'humiliation', 'hurt', 'hysteria', 'infatuation', 'insecurity', 'insult', 'irritation', 'isolation', 'jealousy', 'jolliness', 'joviality', 'joy', 'jubilation', 'liking', 'loathing', 'loneliness', 'longing', 'love', 'lust', 'melancholy', 'misery', 'mortification', 'neglect', 'nervousness', 'optimism', 'outrage', 'panic', 'passion', 'pity', 'pleasure', 'pride', 'rage', 'rapture', 'regret', 'rejection', 'relief', 'remorse', 'resentment', 'revulsion', 'sadness', 'satisfaction', 'scorn', 'sentimentality', 'shame', 'shock', 'sorrow', 'spite', 'suffering', 'surprise', 'sympathy', 'tenderness', 'tenseness', 'terror', 'thrill', 'torment', 'triumph', 'uneasiness', 'unhappiness', 'vengefulness', 'woe', 'worry', 'wrath', 'zeal', 'zest']
label_to_idx = {emotion: idx for idx, emotion in enumerate(emotion_list)}

primary_emotion_mapping = {'adoration': 'Love','affection': 'Love','aggravation': 'Anger','agitation': 'Fear','agony': 'Sadness','alarm': 'Fear','alienation': 'Sadness','amazement': 'Surprise','amusement': 'Happiness','anger': 'Anger','anguish': 'Sadness','annoyance': 'Anger','anxiety': 'Fear','apprehension': 'Fear','arousal': 'Desire','astonishment': 'Surprise','attraction': 'Love','bitterness': 'Anger','bliss': 'Happiness','caring': 'Love','cheerfulness': 'Happiness','compassion': 'Love','contempt': 'Disgust','contentment': 'Happiness','defeat': 'Sadness','dejection': 'Sadness','delight': 'Happiness','depression': 'Sadness','desire': 'Desire','despair': 'Sadness','disappointment': 'Sadness','disgust': 'Disgust','dislike': 'Disgust','dismay': 'Sadness','displeasure': 'Disgust','distress': 'Sadness','dread': 'Fear','eagerness': 'Desire','ecstasy': 'Happiness','elation': 'Happiness','embarrassment': 'Fear','enjoyment': 'Happiness','enthrallment': 'Happiness','enthusiasm': 'Happiness','envy': 'Anger','euphoria': 'Happiness','exasperation': 'Anger','excitement': 'Happiness','exhilaration': 'Happiness','fear': 'Fear','ferocity': 'Anger','fondness': 'Love','fright': 'Fear','frustration': 'Anger','fury': 'Anger','gaiety': 'Happiness','gladness': 'Happiness','glee': 'Happiness','gloom': 'Sadness','glumness': 'Sadness','grief': 'Sadness','grouchiness': 'Anger','grumpiness': 'Anger','guilt': 'Sadness','happiness': 'Happiness','hate': 'Anger','homesickness': 'Sadness','hope': 'Happiness','hopelessness': 'Sadness','horror': 'Fear','hostility': 'Anger','humiliation': 'Sadness','hurt': 'Sadness','hysteria': 'Fear','infatuation': 'Love','insecurity': 'Fear','insult': 'Anger','irritation': 'Anger','isolation': 'Sadness','jealousy': 'Anger','jolliness': 'Happiness','joviality': 'Happiness','joy': 'Happiness','jubilation': 'Happiness','liking': 'Love','loathing': 'Disgust','loneliness': 'Sadness','longing': 'Desire','love': 'Love','lust': 'Desire','melancholy': 'Sadness','misery': 'Sadness','mortification': 'Sadness','neglect': 'Sadness','nervousness': 'Fear','optimism': 'Happiness','outrage': 'Anger','panic': 'Fear','passion': 'Desire','pity': 'Love','pleasure': 'Happiness','pride': 'Happiness','rage': 'Anger','rapture': 'Happiness','regret': 'Sadness','rejection': 'Sadness','relief': 'Happiness','remorse': 'Sadness','resentment': 'Anger','revulsion': 'Disgust','sadness': 'Sadness','satisfaction': 'Happiness','scorn': 'Disgust','sentimentality': 'Love','shame': 'Sadness','shock': 'Surprise','sorrow': 'Sadness','spite': 'Anger','suffering': 'Sadness','surprise': 'Surprise','sympathy': 'Love','tenderness': 'Love','tenseness': 'Fear','terror': 'Fear','thrill': 'Happiness','torment': 'Sadness','triumph': 'Happiness','uneasiness': 'Fear','unhappiness': 'Sadness','vengefulness': 'Anger','woe': 'Sadness','worry': 'Fear','wrath': 'Anger','zeal': 'Happiness','zest': 'Happiness'}

primary_emotions = ['Love', 'Anger', 'Fear', 'Sadness', 'Happiness', 'Surprise', 'Desire', 'Disgust']
primary_emotion_to_idx = {emotion: idx for idx, emotion in enumerate(primary_emotions)}

if not os.path.exists(compiled_labels_path):
    json_file_path = 'training_raw_data.json'
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    emotion_list = ['adoration', 'affection', 'aggravation', 'agitation', 'agony', 'alarm', 'alienation', 'amazement', 'amusement', 'anger', 'anguish', 'annoyance', 'anxiety', 'apprehension', 'arousal', 'astonishment', 'attraction', 'bitterness', 'bliss', 'caring', 'cheerfulness', 'compassion', 'contempt', 'contentment', 'defeat', 'dejection', 'delight', 'depression', 'desire', 'despair', 'disappointment', 'disgust', 'dislike', 'dismay', 'displeasure', 'distress', 'dread', 'eagerness', 'ecstasy', 'elation', 'embarrassment', 'enjoyment', 'enthrallment', 'enthusiasm', 'envy', 'euphoria', 'exasperation', 'excitement', 'exhilaration', 'fear', 'ferocity', 'fondness', 'fright', 'frustration', 'fury', 'gaiety', 'gladness', 'glee', 'gloom', 'glumness', 'grief', 'grouchiness', 'grumpiness', 'guilt', 'happiness', 'hate', 'homesickness', 'hope', 'hopelessness', 'horror', 'hostility', 'humiliation', 'hurt', 'hysteria', 'infatuation', 'insecurity', 'insult', 'irritation', 'isolation', 'jealousy', 'jolliness', 'joviality', 'joy', 'jubilation', 'liking', 'loathing', 'loneliness', 'longing', 'love', 'lust', 'melancholy', 'misery', 'mortification', 'neglect', 'nervousness', 'optimism', 'outrage', 'panic', 'passion', 'pity', 'pleasure', 'pride', 'rage', 'rapture', 'regret', 'rejection', 'relief', 'remorse', 'resentment', 'revulsion', 'sadness', 'satisfaction', 'scorn', 'sentimentality', 'shame', 'shock', 'sorrow', 'spite', 'suffering', 'surprise', 'sympathy', 'tenderness', 'tenseness', 'terror', 'thrill', 'torment', 'triumph', 'uneasiness', 'unhappiness', 'vengefulness', 'woe', 'worry', 'wrath', 'zeal', 'zest']

    label_to_idx = {emotion: idx for idx, emotion in enumerate(emotion_list)}

    labels = []

    for image in data:
        labels.append(label_to_idx[image['label']])

    df = pd.DataFrame()
    df['Label'] = labels
    df['Name'] = [str(i) + ".jpg" for i in range(len(df))]

    valid_images = []
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Checking image paths"):
        img_path = os.path.join(img_dir, row['Name'])
        if os.path.exists(img_path):
            valid_images.append(idx)

    df = df.iloc[valid_images].reset_index(drop=True)
    df.to_csv(compiled_labels_path, index=False)
else:
    df = pd.read_csv(compiled_labels_path)

df['Label'] = df['Label'].apply(lambda x: primary_emotion_to_idx[primary_emotion_mapping[emotion_list[x]]])

print(df['Label'].value_counts())

Label
4    176953
3     89365
1     67761
2     65732
0     47367
5     38096
6     20224
7     11428
Name: count, dtype: int64


In [12]:
class EmotionDataset(Dataset):
    @staticmethod
    def get_transforms(is_training: bool = False) -> transforms.Compose:
        if is_training:
            return transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomRotation(15),
                transforms.RandomAffine(
                    degrees=0,
                    translate=(0.1, 0.1),
                    scale=(0.9, 1.1)
                ),
                transforms.ColorJitter(
                    brightness=0.2,
                    contrast=0.2,
                    saturation=0.2,
                    hue=0.1
                ),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]
                )
            ])
        else:
            return transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]
                )
            ])
    
    def __init__(
        self,
        df: pd.DataFrame,
        img_dir: str,
        transform: Optional[transforms.Compose] = None,
        is_training: bool = False
    ):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform if transform is not None else self.get_transforms(is_training)
    
    def __len__(self) -> int:
        return len(self.df)
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, int]:
        try:
            img_path = os.path.join(self.img_dir, self.df.iloc[idx]['Name'])
            image = Image.open(img_path).convert('RGB')
            
            if self.transform:
                image = self.transform(image)
            
            label = self.df.iloc[idx]['Label']
            return image, label
            
        except Exception as e:
            print(f"Error loading image {img_path}: {str(e)}")
            return torch.zeros((3, 224, 224)), -1

def create_data_loaders(
    df: pd.DataFrame,
    img_dir: str,
    batch_size: int = 32,
    train_ratio: float = 0.8,
    num_workers: int = 4
):
    from torch.utils.data import DataLoader, random_split
    
    train_size = int(train_ratio * len(df))
    val_size = len(df) - train_size
    train_df = df.iloc[:train_size].reset_index(drop=True)
    val_df = df.iloc[train_size:].reset_index(drop=True)
    
    train_dataset = EmotionDataset(
        df=train_df,
        img_dir=img_dir,
        is_training=True
    )
    
    val_dataset = EmotionDataset(
        df=val_df,
        img_dir=img_dir,
        is_training=False
    )
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )
    
    return train_loader, val_loader

class EmotionClassifier(nn.Module):
    def __init__(self, num_classes: int = 8, pretrained: bool = True):
        super(EmotionClassifier, self).__init__()
        self.resnet = models.resnet50(pretrained=pretrained)
        
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
        for param in self.resnet.parameters():
            param.requires_grad = False
        
        for param in self.resnet.layer3.parameters():
            param.requires_grad = True
        for param in self.resnet.layer4.parameters():
            param.requires_grad = True
        for param in self.resnet.fc.parameters():
            param.requires_grad = True

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.resnet(x)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}")

train_loader, val_loader = create_data_loaders(
    df=df,
    img_dir='./faces',
    batch_size=128
)

Using device cuda


In [ ]:
model = EmotionClassifier(num_classes=8)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

best_acc = 21.0
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
    running_loss = 0.0
    
    for batch_idx, (images, labels) in enumerate(train_pbar):
        if -1 in labels:
            continue
            
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss = (running_loss * batch_idx + loss.item()) / (batch_idx + 1)
        train_pbar.set_postfix({'loss': f'{running_loss:.4f}'})
    
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Valid]')
        
        for batch_idx, (images, labels) in enumerate(val_pbar):
            if -1 in labels:
                continue
                
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            val_loss = (val_loss * batch_idx + loss.item()) / (batch_idx + 1)
            val_pbar.set_postfix({
                'loss': f'{val_loss:.4f}',
                'acc': f'{100 * correct / total:.2f}%'
            })
        
        if epoch == 0 or (100 * correct / total) > best_acc:
            best_acc = 100 * correct / total
            with open('best_model-8classes.pkl', 'wb') as f:
                torch.save(model.state_dict(), f)
            print(f'Saving best model with accuracy {best_acc:.2f}%')
    
    print(f'Epoch {epoch+1}/{num_epochs}:')
    print(f'Training Loss: {running_loss:.4f}')
    print(f'Validation Loss: {val_loss:.4f}')
    print(f'Validation Accuracy: {100 * correct / total:.2f}%\n')

/export/home/acs/stud/c/cconstantinescu2705/.conda/envs/llm/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/export/home/acs/stud/c/cconstantinescu2705/.conda/envs/llm/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/20 [Valid]: 100%|████████████████████████████████████| 808/808 [51:30<00:00,  3.83s/it, loss=1.4269, acc=50.44%]


Saving best model with accuracy 50.44%
Epoch 1/20:
Training Loss: 1.3409
Validation Loss: 1.4269
Validation Accuracy: 50.44%



Epoch 2/20 [Train]:  26%|███████████                                | 829/3231 [1:00:25<2:07:27,  3.18s/it, loss=1.2688]